In [1]:
%cd ../

/Users/harrysonghurst/Documents/Coursework/L48 Machine Learning and the Physical World/TrafficEmu


In [2]:
# If you get a SciPy error when installing Emukit, build it from source:

# git clone https://github.com/amzn/Emukit.git
# cd Emukit
# pip install -r requirements/requirements.txt
# python setup.py develop

In [3]:
import numpy as np
import pickle
import emukit as ek
import GPy

from emukit.model_wrappers import GPyModelWrapper
from emukit.experimental_design.experimental_design_loop import ExperimentalDesignLoop
from emukit.core import ParameterSpace, ContinuousParameter, DiscreteParameter
from emukit.core.initial_designs import RandomDesign, latin_design
from emukit.experimental_design.acquisitions import ModelVariance, IntegratedVarianceReduction
from emukit.core.loop import UserFunctionWrapper
from emukit.bayesian_optimization.acquisitions import ExpectedImprovement
from emukit.bayesian_optimization.loops import BayesianOptimizationLoop
from emukit.core.optimization import GradientAcquisitionOptimizer
import matplotlib.pyplot as plt
import seaborn as sns

from sumo_grid_simulation.grid_simulation import Simulator
import experimental_design.config as config

sns.set()

### Pseudocode

Fitting a gaussian process to a simulator using Emukit takes the following form (taken from L48 lectures):

```
initialize GP with some randomly chosen points
while stopping condition is not met:
    compute candidate point(s) using GP and acquisition funciton (model_variance) -> new point
    evaluate this new point with our simulator/user function -> observation
    update model with new observation -> new GP
```

### Simulator

In [4]:
simulator = Simulator(end_time=300)

### User Function
 
This is the function we want to understand. Namedly, how are CO2 emissions and timeLoss (the time lost due to driving below the ideal speed) affected by the following parameters: 

```
gridSize: the size of the grid network where the simulation is carried out
edgeMaxSpeed: legal speed limit in m/s - this can be exceeded (11.11 == 40km/h)
maxSpeed: the absolute maximum velocity of any vehicle in m/s (55.55 == 200 km/h)
edgeLength: length of the roads between intersections in meters
numLanes: number of lanes per road
accel: The acceleration ability of vehicles in m/s^2.
```

*A complete list of parmeters analysed is discussed in our report.*

In [5]:
def user_function(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.2
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['routeLength'])
        i += 1
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

### Model Inputs / parameter space

In [6]:
gridSize = DiscreteParameter('gridSize', domain=[3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20])
edgeMaxSpeed = ContinuousParameter('edgeMaxSpeed', min_value=8, max_value=19)
edgeLength = ContinuousParameter('edgeLength', min_value=30, max_value=70)
numberOfLanes = DiscreteParameter('numberOfLanes', domain=[1,2,3])
accel = ContinuousParameter('accel', 1.5, 5.)

parameter_space = ParameterSpace([gridSize, edgeMaxSpeed, edgeLength, numberOfLanes, accel])

### Get init and test points

In [7]:
design = RandomDesign(parameter_space)
init_X = design.get_samples(20)
init_Y = user_function(init_X)

design = RandomDesign(parameter_space)
test_X = design.get_samples(100)
test_Y = user_function(test_X)

[[16.         14.26673765 42.26563219  1.          3.3250345 ]
 [ 5.         11.99550728 53.22371287  2.          1.84254406]
 [10.         13.42157625 47.8207475   3.          2.62849847]
 [19.         10.99085753 37.5983005   3.          1.64518036]
 [ 5.          9.99053082 43.83530281  2.          3.18764036]
 [14.         10.6573746  32.92800549  2.          3.24687241]
 [16.          8.96021016 52.85024401  3.          3.31964014]
 [15.         16.38373519 38.66565652  3.          3.77951955]
 [20.         12.63518694 40.46642563  3.          4.42391479]
 [20.          9.38254641 37.95138823  1.          2.45156313]
 [14.         10.67463094 40.85488606  2.          3.61766387]
 [ 8.         18.94882678 57.57934457  3.          2.8185991 ]
 [17.         16.66219944 36.6487441   1.          2.38244049]
 [ 4.         15.50793326 41.86086832  3.          4.45706677]
 [19.          8.47763026 59.05202552  1.          4.22648744]
 [12.         10.95792113 47.53915507  2.          1.55

 Retrying in 1 seconds

Evaluating input: 2 of 100

 Retrying in 1 seconds

Evaluating input: 3 of 100

 Retrying in 1 seconds

Evaluating input: 4 of 100

 Retrying in 1 seconds

Evaluating input: 5 of 100

 Retrying in 1 seconds

Evaluating input: 6 of 100

 Retrying in 1 seconds

Evaluating input: 7 of 100

 Retrying in 1 seconds

Evaluating input: 8 of 100

 Retrying in 1 seconds

Evaluating input: 9 of 100

 Retrying in 1 seconds

Evaluating input: 10 of 100

 Retrying in 1 seconds

Evaluating input: 11 of 100

 Retrying in 1 seconds


error: unpack requires a buffer of 4 bytes

### Experimental Design - Model Variance

In [15]:
MV_Xs = []
MV_Ys = []
mses_MV = []
    
def user_function_model_variance(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.2
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['routeLength'])
        MV_Xs.append([int(gridSize), edgeMaxSpeed, edgeLength, int(numLanes), accel])
        MV_Ys.append(CO2/s['routeLength'])

        i += 1
    
    # evaluate / validate model
    if len(MV_Xs) % 50 == 0:
        pred_Y, pred_Y_variance = emukit_model_mv.predict(np.array(test_X))
        rmse = np.sqrt(np.mean((pred_Y-test_Y)**2))
        mses_MV.append((len(MV_Xs), rmse))
        print('EVALUATED: ', len(MV_Xs), rmse)
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

#### Model / Emulator (GP)

Our surrogate model is our emulator. In this case, a gaussian process.

In [16]:
emulator_mv = GPy.models.GPRegression(init_X, init_Y)
emukit_model_mv = GPyModelWrapper(emulator_mv, n_restarts=5)
emukit_model_mv.optimize()
emulator_mv

Optimization restart 1/5, f = -67.57915497275103
Optimization restart 2/5, f = -38.643089255789256
Optimization restart 3/5, f = -38.64309508277225
Optimization restart 4/5, f = -63.23622460617891
Optimization restart 5/5, f = -38.64308921188966


GP_regression.,value,constraints,priors
rbf.variance,0.0032787112232347112,+ve,
rbf.lengthscale,74.77079087562171,+ve,
Gaussian_noise.variance,2.7949890384605055e-05,+ve,


#### Acquisition Function

In [17]:
model_variance = ModelVariance(model=emukit_model_mv)

#### Optimizer

In [18]:
optimizer = GradientAcquisitionOptimizer(parameter_space)


#### Experimental Design Loop

In [19]:
ed_loop_mv = ExperimentalDesignLoop(
    model = emukit_model_mv,
    space = parameter_space,
    acquisition = model_variance,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [20]:
ed_loop_mv.run_loop(user_function_model_variance, 750)

Optimization restart 1/5, f = -67.57915497277844
Optimization restart 2/5, f = -38.643089211701636
Optimization restart 3/5, f = -38.64308925045841
Optimization restart 4/5, f = -67.5791549601593
Optimization restart 5/5, f = -38.643089211701685
[[20.         20.56323278 30.45933824  1.          4.87437208]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -71.25737182232857
Optimization restart 2/5, f = -40.15188532238996
Optimization restart 3/5, f = -40.151885322389646
Optimization restart 4/5, f = -40.15192109777739
Optimization restart 5/5, f = -40.15188532238964
[[ 3.         24.5081419  68.85494919  3.          3.53975891]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -74.69204245184355
Optimization restart 2/5, f = -74.69204245164534
Optimization restart 3/5, f = -74.69204241404252
Optimization rest

 Retrying in 1 seconds
Optimization restart 1/5, f = -137.4437354259297
Optimization restart 2/5, f = -137.44537098967174
Optimization restart 3/5, f = -137.445370989698
Optimization restart 4/5, f = -137.44537098755308
Optimization restart 5/5, f = -137.44537098873957
[[20.         22.61826022 30.62750479  2.          1.57330998]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -141.5702970744477
Optimization restart 2/5, f = -141.57032177416892
Optimization restart 3/5, f = -141.57032177352747
Optimization restart 4/5, f = -72.09702091066825
Optimization restart 5/5, f = -141.57032177412543
[[ 4.         24.67244448 69.38597536  1.          1.59530008]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -143.3070664802919
Optimization restart 2/5, f = -143.3070664764629
Optimization restart 3/5, f = -143.30706

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


 Retrying in 1 seconds
Optimization restart 1/5, f = -207.26317895805624
Optimization restart 2/5, f = -103.10461474101852
Optimization restart 3/5, f = -207.2631789438312
Optimization restart 4/5, f = -207.26317895454957
Optimization restart 5/5, f = -207.2631789594652
[[ 3.          8.14156709 46.77647724  3.          1.51668114]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -210.87857017381904
Optimization restart 2/5, f = -210.87859499055833
Optimization restart 3/5, f = -210.87859498693533
Optimization restart 4/5, f = -210.87859499272585
Optimization restart 5/5, f = -105.07704152812059
[[ 3.         24.57393824 54.81421182  1.          4.85926593]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -214.76491056173657
Optimization restart 2/5, f = -214.7649105637943
Optimization restart 3/5, f = -214.7

 Retrying in 1 seconds
Optimization restart 1/5, f = -284.7326289635124
Optimization restart 2/5, f = -284.73262896351054
Optimization restart 3/5, f = -284.7326289635235
Optimization restart 4/5, f = -284.732628948966
Optimization restart 5/5, f = -284.73262896308927
[[14.         24.86756115 63.61651136  1.          4.84335402]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -289.2406488319132
Optimization restart 2/5, f = -289.2406488358574
Optimization restart 3/5, f = -289.24064882967286
Optimization restart 4/5, f = -289.2406488351182
Optimization restart 5/5, f = -289.2406488358509
[[ 9.         24.50792    30.3926305   3.          4.69822985]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -293.64395867562246
Optimization restart 2/5, f = -293.6439586755876
Optimization restart 3/5, f = -293.6439586

...

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square


Optimization restart 2/5, f = -1063.104877055353
Optimization restart 3/5, f = -1063.1048770551083
Optimization restart 4/5, f = -1063.1048770609432
Optimization restart 5/5, f = -1063.1048770586633
[[20.          8.56377208 65.49573982  3.          1.73858759]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -1067.7934731658538
Optimization restart 2/5, f = -1067.7935218881116
Optimization restart 3/5, f = -1067.793521950979
Optimization restart 4/5, f = -1067.7935219479073
Optimization restart 5/5, f = -1067.7935219299986
[[ 3.          8.65813472 30.39135671  3.          1.54158478]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -1072.0048013523399
Optimization restart 2/5, f = -1072.0048013646294
Optimization restart 3/5, f = -1072.0048013668747
Optimization restart 4/5, f = -1072.0048013653932
Optimiza

 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/stationary.py:168: RuntimeWarning:overflow encountered in true_divide
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:52: RuntimeWarning:overflow encountered in square
 /Users/harrysonghurst/opt/anaconda3/envs/traffic/lib/python3.8/site-packages/GPy/kern/src/rbf.py:76: RuntimeWarning:invalid value encountered in multiply


Optimization restart 2/5, f = -1116.6369500802298
Optimization restart 3/5, f = -1121.2376424836086
Optimization restart 4/5, f = -1121.2376424718284
Optimization restart 5/5, f = -1121.2376423453982
[[10.         24.90472621 69.86110132  3.          4.34367545]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -1125.9437790476866
Optimization restart 2/5, f = -1125.943919880615
Optimization restart 3/5, f = -1125.9439198761117
Optimization restart 4/5, f = -1125.9439198817008
Optimization restart 5/5, f = -1125.9439198802093
[[20.          8.07545063 37.44171321  1.          4.42335929]]

User function time loss called with 1 inputs to simulate

Evaluating input: 1 of 1

 Retrying in 1 seconds
Optimization restart 1/5, f = -1129.3588246140398
Optimization restart 2/5, f = -1129.3588246332981
Optimization restart 3/5, f = -1129.3588244035068
Optimization restart 4/5, f = -1129.3588246270217
Optimiz

KeyboardInterrupt: 

### Experimental Design - Integrated Variance Reduction

In [ ]:
IVR_Xs = []
IVR_Ys = []
mses_IVR = []
    
def user_function_integrated_variance_reduction(X):
    """  X = inputs - emukit doesnt pass named args, just an NxM ndarray, N is the number of points to evaluate, M is the number of parameters per each point """
    result = []
    i = 0
    
    print(X)
    print(f'\nUser function time loss called with {X.shape[0]} inputs to simulate')

    
    for gridSize, edgeMaxSpeed, edgeLength, numLanes, accel in X:
        print(f'\nEvaluating input: {i+1} of {X.shape[0]}\n')
        
        alpha = 0.2
        max_number_of_vehicles = ((gridSize - 1) * gridSize * 2 + 4 * gridSize) * edgeLength / 5
        period = 300/(max_number_of_vehicles * alpha)
        
        s = simulator.simulate(
            gridSize      = int(gridSize),
            edgeMaxSpeed  = edgeMaxSpeed,
            edgeLength    = edgeLength,
            numberOfLanes = int(numLanes),
            accel         = accel,
            trips_generator_period = period
        )
        # average co2 / average route duration
        CO2 = s['CO2']/s['num_emissions_samples']
        # average emissions / average route length
        result.append(CO2/s['routeLength'])
        IVR_Xs.append([int(gridSize), edgeMaxSpeed, edgeLength, int(numLanes), accel])
        IVR_Ys.append(CO2/s['routeLength'])

        i += 1
    
    # evaluate / validate model
    if len(IVR_Xs) % 50 == 0:
        pred_Y, pred_Y_variance = emukit_model_ivr.predict(np.array(test_X))
        rmse = np.sqrt(np.mean((pred_Y-test_Y)**2))
        mses_IVR.append((len(IVR_Xs), rmse))
        print('EVALUATED: ', len(IVR_Xs), rmse)
        
    # expand dims is essential or the acquition function breaks
    return np.expand_dims(np.array(result), 1)  

#### Model / Emulator (GP)

In [ ]:
emulator_ivr = GPy.models.GPRegression(init_X, init_Y)
emukit_model_ivr = GPyModelWrapper(emulator_ivr, n_restarts=5)
emukit_model_ivr.optimize()

#### Acquisition Function

In [ ]:
integrated_variance_reduction = IntegratedVarianceReduction(
    model=emukit_model_ivr, 
    space=parameter_space, 
    num_monte_carlo_points=100
)

#### Optimizer

In [ ]:
optimizer = GradientAcquisitionOptimizer(parameter_space)

#### Experimental Design Loop

In [ ]:
ed_loop_ivr = ExperimentalDesignLoop(
    model = emukit_model_ivr,
    space = parameter_space,
    acquisition = integrated_variance_reduction,
    acquisition_optimizer = optimizer,
    batch_size = 1
)

In [ ]:
ed_loop_ivr.run_loop(user_function_integrated_variance_reduction, 750)

### Random GP

Supervised learning

In [ ]:
# Train points are used to optimize a GP without experimental design - to verify experimental design works.

design = RandomDesign(parameter_space)
train_X = design.get_samples(770)
train_Y = user_function_time_loss(train_X)

In [ ]:
emulator_rnd = GPy.models.GPRegression(train_X, train_Y)
emukit_model_rnd = GPyModelWrapper(emulator_rnd, n_restarts=5)
emukit_model_rnd.optimize()
emulator_rnd

In [ ]:
mses_RND = []
for j in range(50,1000,50):
    xx = train_X[:j]
    yy = train_Y[:j]
    emulator_x = GPy.models.GPRegression(xx, yy)
    emukit_x = GPyModelWrapper(emulator_x, n_restarts=1)
    emukit_x.optimize()
    
    pred_Y, pred_Y_variance = emukit_x.predict(np.array(test_X))
    rmse = np.sqrt(np.mean((pred_Y-test_Y)**2))
    mses_RND.append((j, rmse))
    print('EVALUATED: ', j, rmse)

### Evaluate

Evaluate RMSE of each model on our test set of 50 randomly selected points.

In [ ]:
pred_Y, pred_Y_variance = emukit_model_ed.predict(test_X)
mse_ed = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Model Variance RMSE: ', mse_ed)

In [ ]:
pred_Y, pred_Y_variance = emukit_model_ird.predict(test_X)
mse_ird = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Experimental Design w/ Integrated Variance Reduction RMSE: ', mse_ird)

In [ ]:
pred_Y, pred_Y_variance = emukit_model_rnd.predict(test_X)
mse_rnd = np.sqrt(np.mean((pred_Y-test_Y)**2))
print('Random GP RMSE: ', mse_rnd)

In [ ]:
p = 1 - (mse_ed/mse_rnd)
f'Experimental Design w/ Model Variance is {p*100:.3f}% better than training on random points.'

In [ ]:
p = 1 - (mse_ird/mse_rnd)
f'Experimental Design w/ Integrated Variance Reduction is {p*100:.3f}% better than training on random points.'

#### Plot MSE Graphs

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
X_mv, Y_mv = zip(*mses_MV)
X_ivr, Y_ivr = zip(*mses_IVR)
X_rnd, Y_rnd = zip(*mses_RND)
ax.plot(X_mv, Y_mv)
ax.plot(X_ivr, Y_ivr)
ax.plot(X_rnd, Y_rnd)

### Save Models

In [ ]:
with open(f'experimental_design/models/model_variance_20_init_points_750_loops_co2_per_distance.pkl', "wb") as f:
     pickle.dump(emukit_model_mv, f)

with open(f'experimental_design/models/integrated_variance_reduction_20_init_points_750_loops_co2_per_distance.pkl', "wb") as f:
     pickle.dump(emukit_model_ivr, f)
        
with open(f'experimental_design/models/770_random_points_supervised_GP_co2_per_distance.pkl', "wb") as f:
     pickle.dump(emukit_model_rnd, f)

### Dump whole session

In [ ]:
import dill                            
dill.dump_session('5_params_co2_per_distance_session.pkl')